In [92]:
# Import 
import pandas as pd
import numpy as np
from scipy import signal
import openpyxl

In [93]:
# Input file
folder_path = r'D:\iriwa\デスクトップ\Code\Project_Python\Research'
file_name = r'\yoshiyama_test.txt'

output_filename = '吉山_test'+'(EEG)'


input_data = pd.read_table(folder_path + file_name, header=None, skiprows=6, names=('time','O1','O2'))
input_data = input_data.interpolate()
print(input_data)

time        O1        O2
0         0.000  1.203125 -0.493750
1         0.001  1.531250 -0.837500
2         0.002  1.884375 -1.096875
3         0.003  1.384375 -1.056250
4         0.004  0.700000 -0.846875
...         ...       ...       ...
182545  182.545  0.696875 -0.012500
182546  182.546  0.978125 -0.425000
182547  182.547  1.093750 -0.759375
182548  182.548  1.234375 -1.046875
182549  182.549  0.878125 -1.115625

[182550 rows x 3 columns]


In [94]:
N  = 1024   # Sampling size # サンプリングサイズ(FFTサイズ)
dt = 0.001  # Sampling period[s]        # サンプリング周期[s]
fs = 1/dt   # Sampling rate[Hz]         # サンプリング周波数[Hz]
TIME = int(len(input_data)/fs)  # Data time[s]  #データ時間[s]
print('%d秒分のデータ'%TIME)
time = pd.DataFrame({'time': [i for i in range(TIME)]})
print(time)

182秒分のデータ
     time
0       0
1       1
2       2
3       3
4       4
..    ...
177   177
178   178
179   179
180   180
181   181

[182 rows x 1 columns]


In [95]:
filter1 = signal.firwin(numtaps=16, cutoff=[1,30],pass_zero=False, fs=1/dt) # Filter (bandpass)
wf = signal.hamming(N)  # Window function(hamming)

In [96]:
output_data = []
for n in range(2):
    filter_data = signal.lfilter(filter1, 1, input_data.iloc[:, n+1])   # Filtering(bandpass)   # フィルタリング(バンドパス)
    # cut_data    = []    #cutting for FFT   # FFT用に整形されたデータ
    # window_data = []    #apply window function # 窓関数適用データ
    
    MPF   = [np.nan for i in range(TIME)]
    alpha = [np.nan for i in range(TIME)]
    beta  = [np.nan for i in range(TIME)]
    theta = [np.nan for i in range(TIME)]
    delta = [np.nan for i in range(TIME)]

    for i in range(TIME):
        # Formatted to 1s(1000+24data) & Apply window function  # 1秒(1000+24個)を切り取り & 窓関数適用
        Format_data = [filter_data[j+i*int(fs)] * wf[j] for j in range(N)]  # Formatted to 1s(1000+24data)  # 1秒(1000+24個)を切り取り

        #Func_FFT() #FFT    #高速フーリエ変換
        F = np.fft.fft(Format_data)
        amplitude = np.abs(F)   # Amplitude     # 振幅スペクトル(F/N/2)
        power = amplitude ** 2        # Power         # パワースペクトル
        density = power/(fs/N)     # Power Density # パワースペクトル密度
        freq = np.fft.fftfreq(N, d=dt)
        f_l = int(len(freq)/2)
        
        #Func_cal
        sum_alpha = 0
        sum_beta = 0
        sum_theta = 0
        sum_delta = 0
        total_fp = 0
        total_power = 0

        for l in range(f_l):
            if freq[l] >= 1.0 and freq[l] <= 30: 
                total_power += amplitude[l]
                total_fp = total_fp + (amplitude[l] * freq[l])
                if freq[l] >= 1.0  and freq[l] <= 4.0:
                    sum_delta += amplitude[l]
                if freq[l] >= 4.0  and freq[l] <= 8.0:
                    sum_theta += amplitude[l]
                if freq[l] >= 8.0  and freq[l] <= 13.0:
                    sum_alpha += amplitude[l]
                if freq[l] >= 13.0 and freq[l] <= 30.0:
                    sum_beta  += amplitude[l]

        MPF[i]   = total_fp  / total_power
        alpha[i] = sum_alpha / total_power 
        beta[i]  = sum_beta  / total_power
        theta[i] = sum_theta / total_power
        delta[i] = sum_delta / total_power
        
    output_data.append(MPF)
    output_data.append(alpha)
    output_data.append(beta)
    output_data.append(theta)
    output_data.append(delta)
df = pd.DataFrame(output_data, index=['O1_MPF', 'O1_alpha', 'O1_beta', 'O1_theta', 'O1_delta',\
                                            'O2_MPF', 'O2_alpha', 'O2_beta', 'O2_theta', 'O2_delta']).T
print(df)

O1_MPF  O1_alpha   O1_beta  O1_theta  O1_delta     O2_MPF  O2_alpha  \
0    13.431973  0.110937  0.513184  0.171229  0.204650   8.542997  0.121766   
1    12.631207  0.166735  0.440731  0.164397  0.228137   9.018923  0.175065   
2    12.538571  0.158664  0.448071  0.142477  0.250788   9.488294  0.220841   
3    11.453838  0.181045  0.393725  0.178593  0.246638   5.445266  0.074890   
4    13.941220  0.138106  0.546005  0.156303  0.159586   5.708028  0.069775   
..         ...       ...       ...       ...       ...        ...       ...   
177   8.625899  0.153632  0.240799  0.158557  0.447012   8.983073  0.130232   
178  12.512118  0.221389  0.417275  0.223861  0.137474   8.794282  0.231354   
179  14.154308  0.231841  0.494551  0.180317  0.093291  10.633854  0.226393   
180  13.495515  0.305092  0.422930  0.152995  0.118983   9.390828  0.212432   
181  11.957704  0.217038  0.381041  0.204213  0.197707  10.378335  0.225247   

      O2_beta  O2_theta  O2_delta  
0    0.217606  0.180619

In [97]:
output = pd.concat([time, df], axis=1)
print(output)

time     O1_MPF  O1_alpha   O1_beta  O1_theta  O1_delta     O2_MPF  \
0       0  13.431973  0.110937  0.513184  0.171229  0.204650   8.542997   
1       1  12.631207  0.166735  0.440731  0.164397  0.228137   9.018923   
2       2  12.538571  0.158664  0.448071  0.142477  0.250788   9.488294   
3       3  11.453838  0.181045  0.393725  0.178593  0.246638   5.445266   
4       4  13.941220  0.138106  0.546005  0.156303  0.159586   5.708028   
..    ...        ...       ...       ...       ...       ...        ...   
177   177   8.625899  0.153632  0.240799  0.158557  0.447012   8.983073   
178   178  12.512118  0.221389  0.417275  0.223861  0.137474   8.794282   
179   179  14.154308  0.231841  0.494551  0.180317  0.093291  10.633854   
180   180  13.495515  0.305092  0.422930  0.152995  0.118983   9.390828   
181   181  11.957704  0.217038  0.381041  0.204213  0.197707  10.378335   

     O2_alpha   O2_beta  O2_theta  O2_delta  
0    0.121766  0.217606  0.180619  0.480009  
1    0.17506

In [98]:
# Output data
output_folder_path = 'D:\\iriwa\\デスクトップ\\Code\\Project_Python\\Research\\EEG_test'
output.to_excel(output_folder_path + '\\' + output_filename +'.xlsx', sheet_name=output_filename, index=False)
